In [1]:
import pandas as pd

df = pd.read_csv('atpbind3d_esm_stats.csv')


In [5]:
metrics = ['mcc', 'micro_auprc', 'sensitivity', 'precision', 'micro_auroc']
df.groupby('model').agg({metric: ['mean', 'std'] for metric in metrics})

mcc           micro_auprc           sensitivity  \
                     mean       std        mean       std        mean   
model                                                                   
esm-t33-gearnet  0.666285  0.015978    0.684233  0.010614    0.617804   

                          precision           micro_auroc            
                      std      mean       std        mean       std  
model                                                                
esm-t33-gearnet  0.035864  0.749175  0.020205    0.921472  0.006905

In [11]:
import pandas as pd
from tabulate import tabulate


def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"


def get_stat_df(filename, verbose=True):
    # Sample loading of your CSV - this would be replaced with your actual loading
    result_df = pd.read_csv(filename)

    metrics = ['micro_auprc', 'mcc',  'sensitivity', 'precision', 'micro_auroc']
    grouped = result_df.groupby('model').agg({
        k: ['mean', 'std'] for k in metrics
    })

    record_counts = result_df.groupby('model').size()

    for metric in metrics:
        grouped[f'{metric}_formatted'] = grouped.apply(
            lambda row: format_mean_std(row, metric), axis=1)

    # Add the record count column to the grouped dataframe
    grouped['record_count'] = record_counts.values

    formatted_df = grouped[[
        k + '_formatted' for k in metrics] + ['record_count']]
    formatted_df.columns = metrics + ['record_count']
    formatted_df = formatted_df.sort_values(metrics[0], ascending=True)

    formatted_df = formatted_df.reset_index()

    if verbose:
        markdown_table = tabulate(formatted_df[['model'] + metrics], headers='keys', tablefmt='pipe', showindex=False)
        print(markdown_table)

    return formatted_df

get_stat_df('atpbind3d_esm_stats.csv')

| model                         | micro_auprc   | mcc           | sensitivity   | precision     | micro_auroc   |
|:------------------------------|:--------------|:--------------|:--------------|:--------------|:--------------|
| esm-t33-gearnet               | 0.684 ± 0.011 | 0.666 ± 0.016 | 0.618 ± 0.036 | 0.749 ± 0.020 | 0.921 ± 0.007 |
| esm-t33-gearnet-adaboost-r10  | 0.703 ± 0.010 | 0.685 ± 0.006 | 0.654 ± 0.021 | 0.745 ± 0.018 | 0.910 ± 0.007 |
| esm-t33-gearnet-resiboost-r10 | 0.712 ± 0.007 | 0.692 ± 0.011 | 0.669 ± 0.025 | 0.744 ± 0.023 | 0.915 ± 0.003 |


,model,micro_auprc,mcc,sensitivity,precision,micro_auroc,record_count
0,esm-t33-gearnet,0.684 ± 0.011,0.666 ± 0.016,0.618 ± 0.036,0.749 ± 0.020,0.921 ± 0.007,5
1,esm-t33-gearnet-adaboost-r10,0.703 ± 0.010,0.685 ± 0.006,0.654 ± 0.021,0.745 ± 0.018,0.910 ± 0.007,5
2,esm-t33-gearnet-resiboost-r10,0.712 ± 0.007,0.692 ± 0.011,0.669 ± 0.025,0.744 ± 0.023,0.915 ± 0.003,5
